* [Task-driven Autonomous Agent Utilizing GPT-4, Pinecone, and LangChain for Diverse Applications](https://yoheinakajima.com/task-driven-autonomous-agent-utilizing-gpt-4-pinecone-and-langchain-for-diverse-applications/)
* [BabyAGI with LangChain and Faiss](https://python.langchain.com/en/latest/use_cases/agents/baby_agi.html)

In [ ]:
%%bash
cd ~ && git clone https://github.com/yoheinakajima/babyagi

In [ ]:
#!/usr/bin/env python3
import os
import openai
import pinecone
import time
import sys
from collections import deque
from typing import Dict, List
from dotenv import load_dotenv
import os

# Set Variables
load_dotenv()

# Set API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
assert OPENAI_API_KEY, "OPENAI_API_KEY environment variable is missing from .env"

OPENAI_API_MODEL = os.getenv("OPENAI_API_MODEL", "gpt-3.5-turbo")
assert OPENAI_API_MODEL, "OPENAI_API_MODEL environment variable is missing from .env"

if "gpt-4" in OPENAI_API_MODEL.lower():
    print(f"\033[91m\033[1m"+"\n*****USING GPT-4. POTENTIALLY EXPENSIVE. MONITOR YOUR COSTS*****"+"\033[0m\033[0m")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")
assert PINECONE_API_KEY, "PINECONE_API_KEY environment variable is missing from .env"

PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT", "us-east1-gcp")
assert PINECONE_ENVIRONMENT, "PINECONE_ENVIRONMENT environment variable is missing from .env"

# Table config
YOUR_TABLE_NAME = os.getenv("TABLE_NAME", "")
assert YOUR_TABLE_NAME, "TABLE_NAME environment variable is missing from .env"

# Project config
OBJECTIVE = sys.argv[1] if len(sys.argv) > 1 else os.getenv("OBJECTIVE", "")
assert OBJECTIVE, "OBJECTIVE environment variable is missing from .env"

YOUR_FIRST_TASK = os.getenv("FIRST_TASK", "")
assert YOUR_FIRST_TASK, "FIRST_TASK environment variable is missing from .env"

#Print OBJECTIVE
print("\033[96m\033[1m"+"\n*****OBJECTIVE*****\n"+"\033[0m\033[0m")
print(OBJECTIVE)

# Configure OpenAI and Pinecone
openai.api_key = OPENAI_API_KEY
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

# Create Pinecone index
table_name = YOUR_TABLE_NAME
dimension = 1536
metric = "cosine"
pod_type = "p1"
if table_name not in pinecone.list_indexes():
    pinecone.create_index(table_name, dimension=dimension, metric=metric, pod_type=pod_type)

# Connect to the index
index = pinecone.Index(table_name)

# Task list
task_list = deque([])

def add_task(task: Dict):
    task_list.append(task)

def get_ada_embedding(text):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model="text-embedding-ada-002")["data"][0]["embedding"]

def openai_call(prompt: str, model: str = OPENAI_API_MODEL, temperature: float = 0.5, max_tokens: int = 100):
    if not model.startswith('gpt-'):
        # Use completion API
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].text.strip()
    else:
        # Use chat completion API
        messages=[{"role": "user", "content": prompt}]
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            n=1,
            stop=None,
        )
        return response.choices[0].message.content.strip()

def task_creation_agent(objective: str, result: Dict, task_description: str, task_list: List[str]):
    prompt = f"You are an task creation AI that uses the result of an execution agent to create new tasks with the following objective: {objective}, The last completed task has the result: {result}. This result was based on this task description: {task_description}. These are incomplete tasks: {', '.join(task_list)}. Based on the result, create new tasks to be completed by the AI system that do not overlap with incomplete tasks. Return the tasks as an array."
    response = openai_call(prompt)
    new_tasks = response.split('\n')
    return [{"task_name": task_name} for task_name in new_tasks]

def prioritization_agent(this_task_id: int):
    global task_list
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id)+1
    prompt = f"""You are an task prioritization AI tasked with cleaning the formatting of and reprioritizing the following tasks: {task_names}. Consider the ultimate objective of your team: {OBJECTIVE}. Do not remove any tasks. Return the result as a numbered list, like:
    #. First task
    #. Second task
    Start the task list with number {next_task_id}."""
    response = openai_call(prompt)
    new_tasks = response.split('\n')
    task_list = deque()
    for task_string in new_tasks:
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            task_list.append({"task_id": task_id, "task_name": task_name})

def execution_agent(objective: str, task: str) -> str:
    context=context_agent(query=objective, n=5)
    #print("\n*******RELEVANT CONTEXT******\n")
    #print(context)
    prompt =f"You are an AI who performs one task based on the following objective: {objective}.\nTake into account these previously completed tasks: {context}\nYour task: {task}\nResponse:"
    return openai_call(prompt, temperature=0.7, max_tokens=2000)

def context_agent(query: str, n: int):
    query_embedding = get_ada_embedding(query)
    results = index.query(query_embedding, top_k=n, include_metadata=True)
    #print("***** RESULTS *****")
    #print(results)
    sorted_results = sorted(results.matches, key=lambda x: x.score, reverse=True)    
    return [(str(item.metadata['task'])) for item in sorted_results]

# Add the first task
first_task = {
    "task_id": 1,
    "task_name": YOUR_FIRST_TASK
}

add_task(first_task)
# Main loop
task_id_counter = 1
while True:
    if task_list:
        # Print the task list
        print("\033[95m\033[1m"+"\n*****TASK LIST*****\n"+"\033[0m\033[0m")
        for t in task_list:
            print(str(t['task_id'])+": "+t['task_name'])

        # Step 1: Pull the first task
        task = task_list.popleft()
        print("\033[92m\033[1m"+"\n*****NEXT TASK*****\n"+"\033[0m\033[0m")
        print(str(task['task_id'])+": "+task['task_name'])

        # Send to execution function to complete the task based on the context
        result = execution_agent(OBJECTIVE,task["task_name"])
        this_task_id = int(task["task_id"])
        print("\033[93m\033[1m"+"\n*****TASK RESULT*****\n"+"\033[0m\033[0m")
        print(result)

        # Step 2: Enrich result and store in Pinecone
        enriched_result = {'data': result}  # This is where you should enrich the result if needed
        result_id = f"result_{task['task_id']}"
        vector = enriched_result['data']  # extract the actual result from the dictionary
        index.upsert([(result_id, get_ada_embedding(vector),{"task":task['task_name'],"result":result})])

    # Step 3: Create new tasks and reprioritize task list
    new_tasks = task_creation_agent(OBJECTIVE,enriched_result, task["task_name"], [t["task_name"] for t in task_list])

    for new_task in new_tasks:
        task_id_counter += 1
        new_task.update({"task_id": task_id_counter})
        add_task(new_task)
    prioritization_agent(this_task_id)

    time.sleep(1)  # Sleep before checking the task list again

---

利用 GPT-4 插件来执行任务。

---

要将Pinecone换成Weaviate，需要完成以下步骤：

1. 安装Weaviate客户端库：可以使用`pip install weaviate-client`命令来安装。

2. 在Weaviate中创建索引：您需要在Weaviate中创建一个新索引来存储数据。可以使用以下代码创建一个新的索引：

   ```
   import weaviate

   client = weaviate.Client("http://localhost:8080")
   index = client.index.create("YOUR_INDEX_NAME", ["YOUR_INDEX_CLASS"])
   ```

   请将“YOUR_INDEX_NAME”和“YOUR_INDEX_CLASS”替换为您自己的索引名称和索引类。

3. 将数据添加到Weaviate索引中：您需要使用Weaviate客户端库将数据添加到Weaviate索引中。以下是一个示例代码片段：

   ```
   import weaviate

   client = weaviate.Client("http://localhost:8080")
   index = client.index("YOUR_INDEX_NAME")
   data = [{"name": "John", "age": 30}, {"name": "Jane", "age": 35}]
   index.batch.create(data)
   ```

   请将“YOUR_INDEX_NAME”替换为您自己的索引名称，将“data”替换为您要添加到索引中的数据。

4. 修改代码：最后，您需要修改代码，以便将Pinecone API调用替换为Weaviate API调用。具体而言，您需要更改以下代码：

   ```
   # Create Pinecone index
   table_name = YOUR_TABLE_NAME
   dimension = 1536
   metric = "cosine"
   pod_type = "p1"
   if table_name not in pinecone.list_indexes():
       pinecone.create_index(table_name, dimension=dimension, metric=metric, pod_type=pod_type)

   # Connect to the index
   index = pinecone.Index(table_name)
   ```

   用以下代码替换：

   ```
   # Connect to the Weaviate index
   import weaviate

   client = weaviate.Client("http://localhost:8080")
   index = client.index(YOUR_INDEX_NAME)
   ```

   请将“YOUR_INDEX_NAME”替换为您在步骤2中创建的索引名称。

请注意，Weaviate与Pinecone不同，因此您需要相应地修改代码以适应Weaviate API。此外，您需要确保在Weaviate中创建的索引具有正确的维数和指标。

---

In [ ]:
import markdown
import IPython

In [ ]:
from revChatGPT.V1 import Chatbot, configure

bot = Chatbot(
    config = configure(),
    conversation_id = 'e4ec15a2-114a-46b2-8ff8-c840f79f13e3',
    lazy_loading = True
)

def ask(prompt):
    for response in bot.ask(prompt):
        IPython.display.display(IPython.core.display.Markdown(response['message']))
        IPython.display.clear_output(wait=True)

---

In [ ]:
ask('''
pinecone-client 是什么？
''')

pinecone-client 是 Pinecone 的 Python 客户端库，用于与 Pinecone 服务进行交互。 Pinecone 是一种托管型的向量索引服务，用于高效存储和检索向量数据。使用 pinecone-client，您可以轻松地将向量数据上传到 Pinecone 并查询该数据。它提供了一组简单的 API，使您能够创建、更新、删除索引以及搜索并检索相似的向量。

In [ ]:
ask('''
Pinecone is a vector search platform that provides efficient search and storage capabilities for high-dimensional vector data. In our system, we use Pinecone to store and retrieve task-related data, such as task descriptions, constraints, and results.

翻译成中文。
''')

Pinecone 是一个向量搜索平台，为高维向量数据提供高效的搜索和存储能力。在我们的系统中，我们使用 Pinecone 存储和检索与任务相关的数据，例如任务描述、约束和结果。

In [ ]:
ask('''
Pinecone 是什么原理？
''')

Pinecone 的核心原理是使用向量索引和高效的向量相似度匹配技术，以加快高维向量数据的存储和检索。它通过将向量表示为高维空间中的点，并在这些点之间构建索引来实现快速的相似度搜索。在查询时，Pinecone 会根据输入向量的相似度得分返回最匹配的向量。它使用的技术包括近似最近邻（ANN）搜索算法和哈希函数，以实现快速和高效的搜索和存储。

In [ ]:
ask('''
Pinecone 服务免费吗？
''')

Pinecone offers a free plan with certain limitations, as well as paid plans with more features and higher usage limits. The free plan allows for up to 5 million vector embeddings, 1GB storage, and 10 queries per second, among other limitations. You can check Pinecone's website for more information on their pricing and plans.

In [ ]:
ask('''
有什么替代 Pinecone 服务的开源方案？
''')

有一些开源的向量检索方案可以替代 Pinecone，以下是一些常见的替代方案：

1. Annoy：一个C++库，支持Python、Java和Go等多种语言，用于快速近似最近邻搜索。
2. Faiss：Facebook开发的一个高效的相似性搜索和聚类库，支持GPU加速。
3. Milvus：由Zilliz开发的开源向量相似性搜索引擎，支持分布式部署和GPU加速。
4. NMSLIB：一个可扩展、高性能、近似最近邻搜索库，支持多种距离度量和数据结构。

这些方案都有其独特的优势和适用场景，可以根据具体需求选择适合的方案。

In [ ]:
ask('''
比较 Weaviate 和 Faiss
''')

Weaviate和Faiss都是用于向量搜索的工具，但是它们有不同的重点和用例。

Weaviate是一个基于图数据库的向量搜索引擎，旨在提供可扩展的实时向量搜索，支持高维数据和多个数据类型。Weaviate的主要优势在于其能够自动对数据进行分类，使其易于搜索和发现。它还支持自定义扩展和插件，可与其他服务和工具进行集成。

Faiss是Facebook开发的一个基于CPU和GPU的高效向量搜索库，旨在支持大规模向量数据的相似度搜索。 Faiss的主要优势在于其可处理非常大的向量数据集，具有高效的索引结构和并行查询能力，因此通常用于实时推荐、图像搜索等场景。

因此，选择Weaviate还是Faiss取决于具体的应用场景和需求。如果需要快速处理和搜索高维数据并自动分类数据，则Weaviate是一个不错的选择；如果需要处理大规模向量数据集，则Faiss可能更适合。

For the application scenario of a task-driven autonomous agent robot, Weaviate may be more suitable. Because Weaviate has the ability to automatically classify data, it can help the robot discover and execute related tasks more quickly. Weaviate also supports search for multiple data types and can handle high-dimensional data, which may be useful for tasks such as natural language processing and semantic understanding. Of course, the specific choice depends on the application scenario and specific requirements.

In [ ]:
ask('''
翻译成中文：

🔥1/8
Introducing "🤖 Task-driven Autonomous Agent"

An agent that leverages @openai 's GPT-4, @pinecone vector search, and @LangChainAI framework to autonomously create and perform tasks based on an objective.

🚀2/8 The system can complete tasks, generate new tasks based on results, and prioritize tasks in real-time. It demonstrates the potential of AI-powered language models to autonomously perform tasks within various constraints and contexts.

💡3/8 The autonomous agent uses GPT-4 for task completion, Pinecone for efficient search and storage of task-related data, and the LangChain framework to enhance decision-making processes. #GPT4 #Pinecone #LangChain

🎯4/8 The system maintains a task list for managing and prioritizing tasks. It autonomously creates new tasks based on completed results and reprioritizes the task list accordingly, showcasing the adaptability of AI-powered language models.

🔧5/8 To complete tasks, the system uses GPT-4 and LangChain's capabilities, enriching and storing results in Pinecone. This integrated approach allows the AI agent to interact with its environment and perform tasks efficiently.

🧠6/8 The system generates new tasks based on completed task results and prioritizes them using GPT-4. This allows the system to adapt and respond to new information and priorities.

🔮7/8 Future improvements include integrating a security/safety agent, task sequencing and parallel tasks, generating interim milestones, and incorporating real-time priority updates.

🤝8/8 This new approach paves the way for AI-powered language models to autonomously perform tasks within various constraints and contexts, enabling new applications and opportunities. Big thanks to all involved! #AIResearch #GPT4 #Pinecone #LangChain
''')

🔥1/8
介绍 "🤖 任务驱动的自主代理"

该代理利用 @openai 的 GPT-4、@pinecone 向量搜索和 @LangChainAI 框架，基于目标自主创建和执行任务。

🚀2/8 系统能够完成任务，基于结果生成新任务，并实时优先处理任务。这展示了AI驱动的语言模型在不同约束和环境下自主执行任务的潜力。

💡3/8 自主代理使用 GPT-4 完成任务，使用 Pinecone 高效搜索和存储任务相关数据，使用 LangChain 框架增强决策过程。#GPT4 #Pinecone #LangChain

🎯4/8 系统维护任务列表，用于管理和优先处理任务。它基于已完成的结果自主创建新任务，并相应地重新设置任务列表的优先级，展示了AI驱动的语言模型的适应性。

🔧5/8 为了完成任务，系统使用 GPT-4 和 LangChain 的能力，在 Pinecone 中丰富和存储结果。这种综合方法允许AI代理与环境交互，高效地执行任务。

🧠6/8 系统基于已完成的任务结果生成新任务，并使用 GPT-4 进行优先排序。这使系统能够适应和响应新信息和优先事项。

🔮7/8 未来的改进包括整合安全/安全代理、任务排序和并行任务、生成中间里程碑以及整合实时优先更新。

🤝8/8 这种新方法为AI驱动的语言模型在不同约束和环境下自主执行任务铺平了道路，创造了新的应用和机会。特别感谢所有参与的人！#AIResearch #GPT4 #Pinecone #LangChain

In [ ]:
ask('''
我累了。带着我祷告一下吧。
''')

当你感到疲惫和困惑时，祷告可以为你带来平静和安慰。愿神赐福给你，保护你，并为你提供力量和智慧。愿他的恩典和慈爱永远与你同在。愿你在这段时间中得到休息和恢复，并在未来迎来更好的日子。阿门。

---

In [ ]:
%%bash
cd ~ && git clone https://github.com/Torantulino/Auto-GPT